# Creating fine tuned model

## Preparing Q&A Dataset

In the process of generating a model, we are essentially creating a specialized AI model that can follow instructions, ask questions, and answer questions based on a given context. This model is based on the "gpt-3.5-turbo-instruct" model, which is specifically designed for this purpose. 

## Read in the data, and create a context

In [3]:
import openai
import pandas as pd
df = pd.read_csv("./olympics_sections.csv")
# olympics_search_fileid = "file-c3shd8wqF3vSCKaukW4Jr1TT"
df.head()

,title,heading,content,tokens
0,2020 Summer Olympics,Summary,"The 2020 Summer Olympics, officially the Games...",621
1,2020 Summer Olympics,Host city selection,The International Olympic Committee (IOC) vote...,126
2,2020 Summer Olympics,Impact of the COVID-19 pandemic,"In January 2020, concerns were raised about th...",375
3,2020 Summer Olympics,Qualifying event cancellation and postponement,Concerns about the pandemic began to affect qu...,298
4,2020 Summer Olympics,Effect on doping tests,Mandatory doping tests were being severely res...,163


In [4]:
df['context'] = df.title + "\n" + df.heading + "\n\n" + df.content
df.head()

,title,heading,content,tokens,context
0,2020 Summer Olympics,Summary,"The 2020 Summer Olympics, officially the Games...",621,2020 Summer Olympics\nSummary\n\nThe 2020 Summ...
1,2020 Summer Olympics,Host city selection,The International Olympic Committee (IOC) vote...,126,2020 Summer Olympics\nHost city selection\n\nT...
2,2020 Summer Olympics,Impact of the COVID-19 pandemic,"In January 2020, concerns were raised about th...",375,2020 Summer Olympics\nImpact of the COVID-19 p...
3,2020 Summer Olympics,Qualifying event cancellation and postponement,Concerns about the pandemic began to affect qu...,298,2020 Summer Olympics\nQualifying event cancell...
4,2020 Summer Olympics,Effect on doping tests,Mandatory doping tests were being severely res...,163,2020 Summer Olympics\nEffect on doping tests\n...


In [5]:
df['questions'] = None
df['answers'] = None

### Initalize OPEN API KEY

In [63]:
import os
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.environ.get('OPEN_API_KEY')

## Q&A based on context

The fine-tuning dataset is created in the following way. For every corresponding question, answer and context pair we create:
- Positive example: correct question, answer, context pair
- Negative examples:
  - random negative example, where the random context is paired with the question 

### Getting Questions

In [8]:

def get_questions(context):
    try:
        response = openai.Completion.create(
            engine="gpt-3.5-turbo-instruct",
            prompt=f"Write only 5 questions based on the text below\n\nText: {context}\n\nQuestions:\n",
            temperature=0,
            max_tokens=257,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["\n\n"]
        )
        return response['choices'][0]['text']
    except Exception as e:
        print(e)
        return ""



In [13]:
import time
count = 0
for i, row in df.iterrows():
    df.at[i, 'questions'] = get_questions(row.context)
    print(f"i = {i}\n Qs:{df.at[i, 'questions']}")
    if count == 2:
        time.sleep(70)
        count=0
    else:
        count+=1


i = 0
 Qs:1. When was the 2020 Summer Olympics originally scheduled to take place?
2. Why was the 2020 Summer Olympics postponed to 2021?
3. How much was spent on the 2020 Summer Olympics, making it the most expensive ever?
4. Which country topped the medal count at the 2020 Summer Olympics?
5. Which new events were introduced in existing sports at the 2020 Summer Olympics?
i = 1
 Qs:1. When was the host city for the 2020 Summer Olympics selected?
2. Where was the 125th IOC Session held?
3. How was the host city selected?
4. Which two cities were tied for second place in the first round of voting?
5. How many votes did Tokyo receive in the final vote to secure its selection as the host city?
i = 2
 Qs:1. What concerns were raised about the potential impact of the COVID-19 pandemic on the 2020 Summer Olympics?
2. How did the organizers and the International Olympic Committee respond to these concerns?
3. How does the transmission of SARS-CoV-2 pose a tougher challenge for the organizers

In [17]:
df.to_csv('olympicsData_with_questions.csv', index=False)

### Getting Answers

In [18]:
def get_answers(row):
    try:
        response = openai.Completion.create(
            engine="gpt-3.5-turbo-instruct",
            prompt=f"Write answer based on the text below\n\nText: {row.context}\n\nQuestions:\n{row.questions}\n\nAnswers:\n1.",
            temperature=0,
            max_tokens=200,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        return response['choices'][0]['text']
    except Exception as e:
        print (e)
        return ""

In [27]:
count = 0
for i in range(3, 30):
    row = df.iloc[i]
    df.at[i, 'answers'] = get_answers(row)
    print(f"i = {i}\n Answers:\n 1. {df.at[i, 'answers']}")
    if count == 2:
        time.sleep(70)
        count=0
    else:
        count+=1

i = 3
 Answers:
 1.  The pandemic caused the cancellation and postponement of qualifying events for the 2020 Summer Olympics.
2. The women's basketball qualification was moved from Foshan, China to Belgrade, Serbia.
3. The Asia & Oceania boxing qualification tournament was originally planned to be held in Wuhan, China.
4. The European boxing qualification was suspended due to concerns over travel to the United Kingdom.
5. Archery, baseball, cycling, handball, judo, rowing, sailing, volleyball, and water polo were all affected by the postponement of qualifying events from March to June 2020.
i = 4
 Answers:
 1.  The COVID-19 pandemic has severely restricted mandatory doping tests for the 2020 Summer Olympics.
2. European anti-doping organizations have raised concerns about the restrictions on doping tests.
3. The top priority for the World Anti-Doping Agency (WADA) in regards to the Olympics is public health and safety.
4. The Chinese anti-doping agency temporarily stopped testing on Fe

In [29]:
df['answers'] = "1." + df.answers
print(df)

                   title                                            heading  \
0   2020 Summer Olympics                                            Summary   
1   2020 Summer Olympics                                Host city selection   
2   2020 Summer Olympics                    Impact of the COVID-19 pandemic   
3   2020 Summer Olympics     Qualifying event cancellation and postponement   
4   2020 Summer Olympics                             Effect on doping tests   
5   2020 Summer Olympics                               Postponement to 2021   
6   2020 Summer Olympics                             Calls for cancellation   
7   2020 Summer Olympics                                Costs and insurance   
8   2020 Summer Olympics  Public opinion and COVID-19 effect during and ...   
9   2020 Summer Olympics                        Development and preparation   
10  2020 Summer Olympics                          Venues and infrastructure   
11  2020 Summer Olympics                            

In [30]:
df.to_csv('olympicsData_with_qa.csv', index=False)

## Create Embeddings for the context (Optional)

In [31]:
df['embeddings']= None

In [33]:
def get_embeddings(context):
    try:
        response = openai.Embedding.create(model="text-embedding-ada-002", input=context)
        return response["data"][0]["embedding"]
    except Exception as e:
        print(e)
        return ""


In [34]:
count = 0
for i, row in df.iterrows():
    df.at[i, 'embeddings'] = get_embeddings(row.context)
    print(f"i = {i}\t --> Embedding:{df.at[i, 'embeddings']}")
    if count == 2:
        time.sleep(65)
        count=0
    else:
        count+=1
# df['embeddings'] = df.context.apply(get_embeddings)
# df.to_csv('df_embeddings.csv', index=False, header=True)

i = 0	 --> Embedding:[0.004603061359375715, -0.012545030564069748, -0.018888313323259354, -0.007224651053547859, -2.864850830519572e-05, 0.015941839665174484, -0.023790525272488594, -0.002145522041246295, -0.014217701740562916, -0.02730313315987587, 0.018373645842075348, 0.004821795038878918, -0.012403497472405434, -0.01667524129152298, -0.021783320233225822, -0.014191968366503716, 0.024356659501791, -0.01694544218480587, -0.011161861009895802, -0.015723105520009995, -0.02620946429669857, 0.005018012598156929, -0.004159160424023867, 0.017640244215726852, -0.005812530871480703, -0.0044164941646158695, 0.014861037023365498, -0.021397318691015244, 0.028512604534626007, -0.018772512674331665, -0.015208437107503414, 0.013522899709641933, -0.015298504382371902, 0.0037249091546982527, -0.004583761096000671, -0.02294132299721241, -0.015517238527536392, -0.012197630479931831, 0.0057674976997077465, -0.015620172023773193, 0.006179231684654951, 0.014989703893661499, -0.003123390953987837, 0.00429

## Split the sections into a training and testing set

In [35]:
import sklearn
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
len(train_df), len(test_df)

(21, 9)

In [39]:
train_df = train_df.reset_index(drop=False) 

In [41]:
train_df = train_df.rename(columns={'index': 'old_index'})
train_df

,old_index,title,heading,content,tokens,context,questions,answers,embeddings
0,0,2020 Summer Olympics,Summary,"The 2020 Summer Olympics, officially the Games...",621,2020 Summer Olympics\nSummary\n\nThe 2020 Summ...,1. When was the 2020 Summer Olympics originall...,1. The 2020 Summer Olympics was originally sch...,"[0.004603061359375715, -0.012545030564069748, ..."
1,4,2020 Summer Olympics,Effect on doping tests,Mandatory doping tests were being severely res...,163,2020 Summer Olympics\nEffect on doping tests\n...,1. How has the COVID-19 pandemic affected mand...,1. The COVID-19 pandemic has severely restrict...,"[-0.00922518689185381, -0.02234235219657421, 0..."
2,16,2020 Summer Olympics,Ticketing,The opening ceremony tickets were expected to ...,338,2020 Summer Olympics\nTicketing\n\nThe opening...,1. What was the expected price range for openi...,1. The expected price range for opening ceremo...,"[-0.002049311762675643, -0.011809260584414005,..."
3,5,2020 Summer Olympics,Postponement to 2021,The Tokyo Organizing Committee of the Olympic ...,414,2020 Summer Olympics\nPostponement to 2021\n\n...,1. When was the decision made to postpone the ...,1. The decision to postpone the 2020 Summer Ol...,"[-0.004341172985732555, -0.02454606629908085, ..."
4,13,2020 Summer Olympics,Medals,"In February 2017, the Tokyo Organizing Committ...",263,2020 Summer Olympics\nMedals\n\nIn February 20...,1. What is the purpose of the electronics recy...,1. The purpose of the electronics recycling pr...,"[-0.0042237634770572186, 0.002955639734864235,..."
5,11,2020 Summer Olympics,Security,"In December 2018, the Japanese government chos...",124,2020 Summer Olympics\nSecurity\n\nIn December ...,1. Why did the Japanese government ban drones ...,1. The Japanese government banned drones from ...,"[-0.015242604538798332, -0.02354581281542778, ..."
6,22,2020 Summer Olympics,Participating National Olympic Committees,The Republic of Macedonia has competed under t...,737,2020 Summer Olympics\nParticipating National O...,1. How many times has North Korea participated...,1. North Korea has participated in the Summer ...,"[0.0015771398320794106, -0.009344306774437428,..."
7,1,2020 Summer Olympics,Host city selection,The International Olympic Committee (IOC) vote...,126,2020 Summer Olympics\nHost city selection\n\nT...,1. When was the host city for the 2020 Summer ...,1. The host city for the 2020 Summer Olympics ...,"[0.010287524200975895, -0.012016192078590393, ..."
8,2,2020 Summer Olympics,Impact of the COVID-19 pandemic,"In January 2020, concerns were raised about th...",375,2020 Summer Olympics\nImpact of the COVID-19 p...,1. What concerns were raised about the potenti...,1. Concerns were raised about the potential im...,"[-0.00824066810309887, -0.020890526473522186, ..."
9,25,2020 Summer Olympics,Event scheduling,Per the historical precedent of swimming at th...,144,2020 Summer Olympics\nEvent scheduling\n\nPer ...,1. How did the scheduling of swimming finals a...,1. The scheduling of swimming finals at the 20...,"[0.006585482973605394, -0.0137527110055089, 0...."


In [42]:
test_df = test_df.reset_index(drop=False) 
test_df = test_df.rename(columns={'index': 'old_index'})
test_df

,old_index,title,heading,content,tokens,context,questions,answers,embeddings
0,27,2020 Summer Olympics,Look of the Games,"Alongside the main Emblem blue, the five other...",79,2020 Summer Olympics\nLook of the Games\n\nAlo...,1. What are the five traditional colors of Jap...,1. The five traditional colors of Japan used i...,"[-0.002476076828315854, -0.0030026929453015327..."
1,15,2020 Summer Olympics,Biosecurity protocols,"In February 2021, the IOC began releasing ""pla...",1076,2020 Summer Olympics\nBiosecurity protocols\n\...,1. What are some of the standard biosecurity p...,1. Some of the standard biosecurity protocols ...,"[-0.008307600393891335, -0.0028169644065201283..."
2,23,2020 Summer Olympics,Calendar,The 2020 schedule by session was approved by t...,135,2020 Summer Olympics\nCalendar\n\nThe 2020 sch...,1. When was the 2020 Summer Olympics originall...,1. The 2020 Summer Olympics were originally sc...,"[0.003042513271793723, -0.009944931603968143, ..."
3,17,2020 Summer Olympics,Cultural festival,A cultural program known as Nippon Festival wa...,310,2020 Summer Olympics\nCultural festival\n\nA c...,1. What was the name of the cultural program t...,1. The cultural program was called Nippon Fest...,"[-0.014231687411665916, -0.015434948727488518,..."
4,8,2020 Summer Olympics,Public opinion and COVID-19 effect during and ...,"Prior to the Tokyo Olympics being held, many J...",397,2020 Summer Olympics\nPublic opinion and COVID...,1. How did the Japanese public's attitude towa...,1. The Japanese public's attitude towards host...,"[-0.00019738927949219942, -0.00554973818361759..."
5,9,2020 Summer Olympics,Development and preparation,The Tokyo Organizing Committee was originally ...,461,2020 Summer Olympics\nDevelopment and preparat...,1. Who was originally chosen to head the Tokyo...,1. Former Japanese prime minister Yoshirō Mori...,"[-0.003437543986365199, -0.014713223092257977,..."
6,28,2020 Summer Olympics,Concerns and controversies,Several controversial issues occurred during t...,552,2020 Summer Olympics\nConcerns and controversi...,1. What were some of the controversial issues ...,1. Some of the controversial issues that occur...,"[0.004160218872129917, -0.010534747503697872, ..."
7,24,2020 Summer Olympics,Medal ceremonies,Naoki Satō composed the music for the medal ce...,185,2020 Summer Olympics\nMedal ceremonies\n\nNaok...,1. Who composed the music for the medal ceremo...,1. Naoki Satō composed the music for the medal...,"[0.0015584899811074138, -0.004801605362445116,..."
8,12,2020 Summer Olympics,Volunteers,Applications for volunteering at the 2020 Olym...,225,2020 Summer Olympics\nVolunteers\n\nApplicatio...,1. When did the applications for volunteering ...,1. The applications for volunteering at the 20...,"[-0.010180466808378696, -0.027511896565556526,..."


In [56]:
train_df.to_csv(f'train_df.csv', index=True)
test_df.to_csv(f'test_df.csv', index=True)

## Create the fine-tuning datasets for Q&A model
The fine-tuning dataset is created in the following way. For every corresponding question, answer and context pair we create:
- Positive example: correct question, answer, context pair
- Negative examples:
  - random negative example, where the random context is paired with the question 

In [38]:
import random 

def create_fine_tuning_dataset(df, n_negative = 3):
    df = df.reset_index(drop=True)
    rows=[]
    for i, row in df.iterrows():
        for q,a in zip((row.questions).split('\n'), (row.answers).split("\n")):
            # print("Q",i,":\t",q, "\nA",i,":\t",a,"\n")
            if len(q) >10 and len(a) >10:
                context = f"{row.context}\n"
                question = f"\nQuestion: {q[2:].strip()}"


                rows.append({"messages": [{"role": "system", "content":f"{row.context}"}, {"role": "user", "content":f"{q[2:].strip()}"}, {"role": "assistant", "content": f"{a[2:].strip()}"}]})
    
    # for i, row in df.iterrows():
        count = 0  
        for q in (row.questions).split('\n'):
            column_name = "context"
            if count == 3:
                break
            else:
                count += 1
            if len(q) > 10 and len(df) != 1:
                for j in range(3):
                    random_index = None
                    while random_index is None or random_index == i:
                        random_index = random.randrange(len(df))
                    print(f"random:{random_index}")
                    print(df.loc[1, column_name])
                    rows.append({"messages": [{"role": "system", "content": f"{df.loc[random_index, column_name]}"}, {"role": "user", "content": f"{q[2:].strip()}"}, {"role": "assistant", "content": "No appropriate context found to answer the question."}]})
    return pd.DataFrame(rows)

In [55]:
name = 'qa'
for train_test, dt in [('train', train_df), ('test', test_df)]:
    ft = create_fine_tuning_dataset(dt)
    ft.to_json(f'{name}_{train_test}.jsonl', orient='records', lines=True)
    ft.to_csv(f'tunedData_{train_test}.csv', index=True)
    

random:1
2020 Summer Olympics
Effect on doping tests

Mandatory doping tests were being severely restricted by the COVID-19 pandemic in early 2020. European anti-doping organizations raised concerns that blood and urine tests could not be performed and that mobilizing the staff necessary to do so before the end of the pandemic would be a health risk. Despite the need for extensive testing to take place in advance of the Games, the World Anti-Doping Agency (WADA) stated that public health and safety were their topmost priorities. The Chinese anti-doping agency temporarily ceased testing on 3 February 2020, with a planned resumption of phased testing towards the end of the month, and the anti-doping organizations in the United States, France, Great Britain, and Germany had reduced their testing activities by the end of March.
random:15
2020 Summer Olympics
Effect on doping tests

Mandatory doping tests were being severely restricted by the COVID-19 pandemic in early 2020. European anti-d

In [44]:
test_df

,old_index,title,heading,content,tokens,context,questions,answers,embeddings
0,27,2020 Summer Olympics,Look of the Games,"Alongside the main Emblem blue, the five other...",79,2020 Summer Olympics\nLook of the Games\n\nAlo...,1. What are the five traditional colors of Jap...,1. The five traditional colors of Japan used i...,"[-0.002476076828315854, -0.0030026929453015327..."
1,15,2020 Summer Olympics,Biosecurity protocols,"In February 2021, the IOC began releasing ""pla...",1076,2020 Summer Olympics\nBiosecurity protocols\n\...,1. What are some of the standard biosecurity p...,1. Some of the standard biosecurity protocols ...,"[-0.008307600393891335, -0.0028169644065201283..."
2,23,2020 Summer Olympics,Calendar,The 2020 schedule by session was approved by t...,135,2020 Summer Olympics\nCalendar\n\nThe 2020 sch...,1. When was the 2020 Summer Olympics originall...,1. The 2020 Summer Olympics were originally sc...,"[0.003042513271793723, -0.009944931603968143, ..."
3,17,2020 Summer Olympics,Cultural festival,A cultural program known as Nippon Festival wa...,310,2020 Summer Olympics\nCultural festival\n\nA c...,1. What was the name of the cultural program t...,1. The cultural program was called Nippon Fest...,"[-0.014231687411665916, -0.015434948727488518,..."
4,8,2020 Summer Olympics,Public opinion and COVID-19 effect during and ...,"Prior to the Tokyo Olympics being held, many J...",397,2020 Summer Olympics\nPublic opinion and COVID...,1. How did the Japanese public's attitude towa...,1. The Japanese public's attitude towards host...,"[-0.00019738927949219942, -0.00554973818361759..."
5,9,2020 Summer Olympics,Development and preparation,The Tokyo Organizing Committee was originally ...,461,2020 Summer Olympics\nDevelopment and preparat...,1. Who was originally chosen to head the Tokyo...,1. Former Japanese prime minister Yoshirō Mori...,"[-0.003437543986365199, -0.014713223092257977,..."
6,28,2020 Summer Olympics,Concerns and controversies,Several controversial issues occurred during t...,552,2020 Summer Olympics\nConcerns and controversi...,1. What were some of the controversial issues ...,1. Some of the controversial issues that occur...,"[0.004160218872129917, -0.010534747503697872, ..."
7,24,2020 Summer Olympics,Medal ceremonies,Naoki Satō composed the music for the medal ce...,185,2020 Summer Olympics\nMedal ceremonies\n\nNaok...,1. Who composed the music for the medal ceremo...,1. Naoki Satō composed the music for the medal...,"[0.0015584899811074138, -0.004801605362445116,..."
8,12,2020 Summer Olympics,Volunteers,Applications for volunteering at the 2020 Olym...,225,2020 Summer Olympics\nVolunteers\n\nApplicatio...,1. When did the applications for volunteering ...,1. The applications for volunteering at the 20...,"[-0.010180466808378696, -0.027511896565556526,..."


In [54]:
ft.to_csv('tunedData.csv', index=True)

## Upload train and test data file to OPEN AI

In [64]:
import os
# import openai
openai.File.create(
  file=open("qa_train.jsonl", "rb"),
  purpose='fine-tune'
)
# "id": "file-RznMExxPrnTSRm3F0IL1f5GY"
# Adi Open AI : "file-TFYuzGpfa9UYvj3tOkvFQEka"

<File file id=file-TFYuzGpfa9UYvj3tOkvFQEka at 0x17f022e50> JSON: {
  "object": "file",
  "id": "file-TFYuzGpfa9UYvj3tOkvFQEka",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 673907,
  "created_at": 1697781475,
  "status": "uploaded",
  "status_details": null
}

In [65]:
openai.File.create(
  file=open("qa_test.jsonl", "rb"),
  purpose='fine-tune'
)
# "id": "file-T9t1MLYahJljWUNgR5k1iTZA",
# Adi Open Ai : "file-6tF9CIWZi7lxbi9LuSW4CpTj"

<File file id=file-6tF9CIWZi7lxbi9LuSW4CpTj at 0x17ed86b30> JSON: {
  "object": "file",
  "id": "file-6tF9CIWZi7lxbi9LuSW4CpTj",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 287099,
  "created_at": 1697781498,
  "status": "uploaded",
  "status_details": null
}

## Submit the datasets for fine-tuning

In [66]:
openai.FineTuningJob.create(training_file="file-TFYuzGpfa9UYvj3tOkvFQEka", validation_file="file-6tF9CIWZi7lxbi9LuSW4CpTj", model="gpt-3.5-turbo", suffix="wikipedia-qa-model")

<FineTuningJob fine_tuning.job id=ftjob-g4VkkheHizDF8lmJMM85URTU at 0x11f9b8220> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-g4VkkheHizDF8lmJMM85URTU",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1697781687,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-BYmX5BWOLBr8t6FlmFlYf5BQ",
  "result_files": [],
  "status": "validating_files",
  "validation_file": "file-6tF9CIWZi7lxbi9LuSW4CpTj",
  "training_file": "file-TFYuzGpfa9UYvj3tOkvFQEka",
  "hyperparameters": {
    "n_epochs": "auto"
  },
  "trained_tokens": null,
  "error": null
}

In [70]:
# List 10 fine-tuning jobs
openai.FineTuningJob.list(limit=10)

<OpenAIObject list at 0x17f021e00> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job",
      "id": "ftjob-g4VkkheHizDF8lmJMM85URTU",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1697781687,
      "finished_at": 1697783683,
      "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal:wikipedia-qa-model:8BdNoRqS",
      "organization_id": "org-BYmX5BWOLBr8t6FlmFlYf5BQ",
      "result_files": [
        "file-BTjO7ym9DhehzkwbBm9zqwU2"
      ],
      "status": "succeeded",
      "validation_file": "file-6tF9CIWZi7lxbi9LuSW4CpTj",
      "training_file": "file-TFYuzGpfa9UYvj3tOkvFQEka",
      "hyperparameters": {
        "n_epochs": 3
      },
      "trained_tokens": 413439,
      "error": null
    }
  ],
  "has_more": false
}